# Linear Modeling

The goal of this lab is to design a class for linear regression, using no classes or functions from Scikit-Learn.

Ensure your class satisfies:
- Includes a class method to fit the model to a Pandas dataframe $X$ and a Pandas series $y$
- Includes a class method to solve for the optimal coefficients
- Includes a class method to make predictions, given a new matrix $\hat{X}$
- Does not invert any matrices explicitly. Instead, solve the normal equations using `np.lingalg.solve`.
- It can be instructed to automatically perform a train-test split and return performance metrics on the test set.
- It can provide metrics including SSE, MSE, RMSE, and $R^2$.

Before programming your class, consider the following questions and record the answers:
- How does your class handle categorical data? How does Sci-kit do it?
- How does your class handle missing data? How does Sci-kit do it?
- Does your class have any methods for creating polynomial expansions or otherwise transforming data? How does Sci-kit do it?
- How does your class handle the bias/intercept/constant? How does Sci-kit do it?
- What output do you automatically provide to the user? Why? How does Sci-kit do it?
- Are you including any tools for statistical inference? How does Sci-kit do it?

In order to measure how long it takes to run code, you can `import time`, and 

```
start = time.time()
<expressions and code>
finish = time.time()
runtime = start-finish
```

For the `heart_hw.csv` and `cars_hw.csv` data in the assignment folder, run some regressions and compare the performance of your class with Sci-Kit's linear regression model. Do you get the same answers for the optimal coefficients, SSE, and $R^2$? Which one runs faster?



In [21]:
import numpy as np
import pandas as pd
import time
import sys
from sklearn.model_selection import train_test_split

class CustomLinearModel:
    def __init__(self, train_test_split_ratio=0.8):
        self.coefficients = None
        self.intercept = None
        self.train_test_split_ratio = train_test_split_ratio
        self.fitted = False

    def fit(self, X, y, split=True):
        if split:
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=self.train_test_split_ratio, random_state=42)
        else:
            X_train, y_train = X, y

        # Intercept col
        X_train = np.c_[np.ones(X_train.shape[0]), X_train]

        # Normal equation using np.linalg.solve
        start = time.time()
        self.coefficients = np.linalg.solve(X_train.T @ X_train, X_train.T @ y_train)
        end = time.time()

        self.intercept = self.coefficients[0]
        self.coefficients = self.coefficients[1:]
        self.fitted = True

        print(f"Trained in {end - start} seconds")

        if split:
            self.evaluate(X_test, y_test)
        
    def predict(self, X):
        if not self.fitted:
            print("Model must be fit before predicting")
            sys.exit()

        # Intercept col
        X = np.c_[np.ones(X.shape[0]), X]
        return X @ np.r_[self.intercept, self.coefficients]
    
    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        SSE = np.sum((y_test - y_pred) ** 2)
        MSE = SSE / len(y_test)
        RMSE = np.sqrt(MSE)
        R2 = 1 - (SSE / np.sum((y_test - np.mean(y_test)) ** 2))
        
        print(f"SSE: {SSE}, MSE: {MSE}, RMSE: {RMSE}, R^2: {R2}")
        return {'SSE': SSE, 'MSE': MSE, 'RMSE': RMSE, 'R2': R2}


In [22]:
# Datasets
heart_df = pd.read_csv("../assignment/data/heart_hw.csv")
cars_df = pd.read_csv("./data/cars_hw.csv")

In [23]:
# Run the custom model on both datasets
X_heart = heart_df.drop(columns=['y']).select_dtypes(include=[np.number])
y_heart = heart_df['y']

X_cars = cars_df.drop(columns=['Price']).select_dtypes(include=[np.number])
y_cars = cars_df['Price']

# Heart dataset
custom_heart_model = CustomLinearModel()
heart_metrics = custom_heart_model.fit(X_heart, y_heart, split=True)

# Cars dataset
custom_cars_model = CustomLinearModel()
cars_metrics = custom_cars_model.fit(X_cars, y_cars, split=True)

Trained in 0.00011706352233886719 seconds
SSE: 2.954182586449766, MSE: 0.14067536125951266, RMSE: 0.375067142335226, R^2: 0.36696087433219293
Trained in 4.291534423828125e-05 seconds
SSE: 15941185397031.21, MSE: 81332578556.2817, RMSE: 285188.67185826594, R^2: 0.29088245916105326


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
def sklearn_regression_evaluation(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

    model = LinearRegression()
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()

    y_pred = model.predict(X_test)
    SSE = np.sum((y_test - y_pred) ** 2)
    MSE = mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(MSE)
    R2 = r2_score(y_test, y_pred)
    runtime = end_time - start_time

    return {'SSE': SSE, 'MSE': MSE, 'RMSE': RMSE, 'R2': R2, 'runtime': runtime}

# Heart dataset
sklearn_heart_metrics = sklearn_regression_evaluation(X_heart, y_heart)

# Cars dataset
sklearn_cars_metrics = sklearn_regression_evaluation(X_cars, y_cars)

sklearn_heart_metrics, sklearn_cars_metrics

({'SSE': np.float64(2.9541825864497664),
  'MSE': 0.1406753612595127,
  'RMSE': np.float64(0.37506714233522603),
  'R2': 0.3669608743321928,
  'runtime': 0.0005919933319091797},
 {'SSE': np.float64(15941185396972.443),
  'MSE': 81332578555.98186,
  'RMSE': np.float64(285188.67185774026),
  'R2': 0.2908824591636674,
  'runtime': 0.00074005126953125})

In [25]:
# The metrics between the custom and sklearn models are extremely similar, with the sklearn one running significantly faster on the 2 datasets. This is most likely due to more efficient implementation of the normal function under the hood.